## Building Predective Models

In [72]:
import numpy as np
import pandas as pd
import os

### Import file

In [73]:
train_test_path=os.path.join(os.path.pardir,'data','processed')
train_path=os.path.join(train_test_path,'train.csv')
test_path=os.path.join(train_test_path,'test.csv')

In [74]:
train_df=pd.read_csv(train_path,index_col='PassengerId')
test_df=pd.read_csv(test_path,index_col='PassengerId')

In [75]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 33 columns):
Survived              891 non-null int64
Age                   891 non-null float64
Fare                  891 non-null float64
FamilySize            891 non-null int64
IsMother              891 non-null int64
Ismale                891 non-null int64
Deck_A                891 non-null int64
Deck_B                891 non-null int64
Deck_C                891 non-null int64
Deck_D                891 non-null int64
Deck_E                891 non-null int64
Deck_F                891 non-null int64
Deck_G                891 non-null int64
Deck_Z                891 non-null int64
Pclass_1              891 non-null int64
Pclass_2              891 non-null int64
Pclass_3              891 non-null int64
Title_Lady            891 non-null int64
Title_Master          891 non-null int64
Title_Miss            891 non-null int64
Title_Mr              891 non-null int64
Title_Mrs             891 non-

In [76]:
train_df.columns

Index(['Survived', 'Age', 'Fare', 'FamilySize', 'IsMother', 'Ismale', 'Deck_A',
       'Deck_B', 'Deck_C', 'Deck_D', 'Deck_E', 'Deck_F', 'Deck_G', 'Deck_Z',
       'Pclass_1', 'Pclass_2', 'Pclass_3', 'Title_Lady', 'Title_Master',
       'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Officer', 'Title_Sir',
       'Fare_Bin_very_low', 'Fare_Bin_low', 'Fare_Bin_high',
       'Fare_Bin_very_high', 'Embarked_C', 'Embarked_Q', 'Embarked_S',
       'AgeState_Adult', 'AgeState_Child'],
      dtype='object')

### Data Prepration

In [77]:
X=train_df.loc[:,'Age':].as_matrix().astype('float')
y=train_df['Survived'].ravel()

In [78]:
print(X.shape)

(891, 32)


In [79]:
print(y.shape)

(891,)


In [80]:
#train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(712, 32) (712,)
(179, 32) (179,)


In [81]:
#Average Survival in train and test
print('mean survival in train {0:.3f}'.format(np.mean(y_train)))
print('mean survival in test {0:.3f}'.format(np.mean(y_test)))

mean survival in train 0.383
mean survival in test 0.385


#### check scikit learn version

In [82]:
import sklearn

In [83]:
sklearn.__version__

'0.19.1'

In [84]:
#make sure v is greater than 0.19.0
#!conda update -y scikit-learn

### Baseline Model

In [85]:
#import function
from sklearn.dummy import DummyClassifier

In [86]:
#create model
model_dummy=DummyClassifier(strategy='most_frequent', random_state=0)

In [87]:
#train_model
model_dummy.fit(X_train,y_train)

DummyClassifier(constant=None, random_state=0, strategy='most_frequent')

In [88]:
print('score of baseline model: {0:.3f}'.format(model_dummy.score(X_test,y_test)))

score of baseline model: 0.615


In [89]:
#Performance metrix
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score


In [90]:
#accuracy score
print('Accuracy of baseline model is: {0:.3f}'.format(accuracy_score(y_test,model_dummy.predict(X_test))))

Accuracy of baseline model is: 0.615


In [91]:
#confusion matrix
print('confusion of baseline model is: {0}'.format(confusion_matrix(y_test,model_dummy.predict(X_test))))

confusion of baseline model is: [[110   0]
 [ 69   0]]


In [92]:
#precision and recall score
print('precision of baseline model is: {0}'.format(precision_score(y_test,model_dummy.predict(X_test))))
print('recall of baseline model is: {0}'.format(recall_score(y_test,model_dummy.predict(X_test))))

precision of baseline model is: 0.0
recall of baseline model is: 0.0


C:\Users\vkrnt\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


### First kaggle submission

In [93]:
#convert to matrix
test_X=test_df.as_matrix().astype('float')

In [94]:
#get predections
predictions=model_dummy.predict(test_X)

In [95]:
df_submission=pd.DataFrame({'PassengerId':test_df.index,'Survived':predictions})

In [96]:
df_submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [97]:
submission_data_path=os.path.join(os.path.pardir,'data','external')
submission_file_path=os.path.join(submission_data_path,'1_dummy.csv')

In [98]:
df_submission.to_csv(submission_file_path,index=False)

In [99]:
def get_submission_file(model,filename):
    test_X=test_df.as_matrix().astype('float')
    predictions=model.predict(test_X)
    df_submission=pd.DataFrame({'PassengerId':test_df.index,'Survived':predictions})
    submission_data_path=os.path.join(os.path.pardir,'data','external')
    submission_file_path=os.path.join(submission_data_path,filename)
    df_submission.to_csv(submission_file_path,index=False)

### logestic regression

In [100]:
from sklearn.linear_model import LogisticRegression

In [101]:
model_lr_1=LogisticRegression(random_state=0)

In [102]:
model_lr_1.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [103]:
print('Score of LR: {0:.3f}'.format(model_lr_1.score(X_test,y_test)))

Score of LR: 0.832


In [104]:
model_lr_1.coef_

array([[-0.02834411,  0.00459056, -0.50301306,  0.68120684, -0.89957053,
         0.12827645, -0.17474497, -0.39225017,  0.53067274,  1.10714647,
         0.40801278, -0.18486252, -0.29507443,  0.9673784 ,  0.49845178,
        -0.33865382,  0.27069309,  1.24499042,  0.50887424, -1.42054802,
         0.97178716,  0.01714598, -0.46576652,  0.16815879,  0.25211553,
         0.28723609,  0.41966594,  0.5013302 ,  0.46819524,  0.15765091,
         0.38241147,  0.74476488]])

## Second Submission

In [105]:
get_submission_file(model_lr_1,'2_lr.csv')

### Hyperparameter Optimization

In [106]:
model_lr=LogisticRegression(random_state=0)

In [107]:
from sklearn.model_selection import GridSearchCV

In [108]:
parameters={'C':[1.0,10.0,50.0,100.0,1000],'penalty':['l1','l2']}
clf=GridSearchCV(model_lr,param_grid=parameters,cv=3)

In [109]:
clf.fit(X_train,y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [1.0, 10.0, 50.0, 100.0, 1000], 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [110]:
clf.best_params_

{'C': 1.0, 'penalty': 'l1'}

In [111]:
print('Best Score is: {0:.2f}'.format(clf.best_score_))

Best Score is: 0.83


### Making third submission

In [112]:
get_submission_file(clf,'3_lr.csv')

## Feature Normalization and Standardisation

In [113]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler

In [114]:
#Feature normalization
scaler=MinMaxScaler()
X_train_scaled=scaler.fit_transform(X_train)

In [115]:
X_train_scaled[:,0].min()

0.0

In [116]:
X_train_scaled[:,0].max()

1.0

In [117]:
#normalise test data
X_test_scaled=scaler.fit_transform(X_test)

## Feature Standarisation

In [118]:
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [119]:
print('Score for  file is: {0:.3f}'.format(clf.score(X_test_scaled,y_test)))

Score for  file is: 0.754


### Model Persistence

In [120]:
#import pickle library
import pickle
import os

In [121]:
#create the file paths
model_file_path=os.path.join(os.path.pardir,'models','model1.pkl')
scaler_file_path=os.path.join(os.path.pardir,'models','scaler1.pkl')

In [122]:
#open the files to write
model_file_pickle=open(model_file_path,'wb')
scaler_file_pickle=open(scaler_file_path,'wb')

In [123]:
#write in file
pickle.dump(clf,model_file_pickle)
pickle.dump(scaler,scaler_file_pickle)

In [124]:
#close the file
model_file_pickle.close()
scaler_file_pickle.close()

## load the persisted file to check if file is working or not

In [125]:
#open files in read mode
model_file_pickle=open(model_file_path,'rb')
scaler_file_pickle=open(scaler_file_path,'rb')

In [126]:
#load files
clf_loaded=pickle.load(model_file_pickle)
scaler_loaded=pickle.load(scaler_file_pickle)
#close files
model_file_pickle.close()
scaler_file_pickle.close()

In [127]:
clf_loaded

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [1.0, 10.0, 50.0, 100.0, 1000], 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [128]:
scaler_loaded

StandardScaler(copy=True, with_mean=True, with_std=True)

In [129]:
#test on test data
X_test_scaled=scaler_loaded.transform(X_test)

In [130]:
print('Score for persisted file is: {0:.3f}'.format(clf_loaded.score(X_test_scaled,y_test)))

Score for persisted file is: 0.754
